In [144]:
#all library connection
import pandas as pd
import numpy as np
import re #регулярки
from math import gcd, isclose
import sympy as sp #библиотека символьных вычеслений, мат.задачи
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [117]:
#pymatgen def to obtain structures
from pymatgen.core.structure import Composition

In [118]:
#loading data
df = pd.read_csv(r'C:\Users\Ксения\Project\Magnet_Exchange_bias\data\Bias_data.csv')

In [119]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 52 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              533 non-null    int64  
 1   crystal_structure(core/shell)                   532 non-null    object 
 2   Space group(core)                               532 non-null    object 
 3   Space group(shell)                              533 non-null    object 
 4   chemical formula                                533 non-null    object 
 5   Structure                                       533 non-null    object 
 6   length (D)                                      521 non-null    float64
 7   width                                           521 non-null    float64
 8   depth                                           509 non-null    float64
 9   shape                                      

In [120]:
#удалим все колонки, значения которых точно не подлежат востановлению
del_col = ['Exp type', 'Scale bar (nm)', 'microphotography pic','Spectrum', 'JCPDS card (core)', 'JCPDS card (shell)', 'X-ray tube (nm)', 'Instrument', 'Crystalline size', 'Material fraction', 'curie temperature (K)', 'H-range min (kOe)', 'Instrument.1', 'M-H curve', 'applied field(kOe)', 'Article id', 'Method', 'H-range min (kOe)','Tb', 'Keff (effective anisotropy constant) (erg/cm3)',
            'Keff (Jm^-3)', 'Total magnetization ', 'Atomic mass ratio', 'Number of magnetic ions per unit cell', 'Name', 'synonyms',
            'get_exchange_group_info', 'cooling fields (mT)', 'Hc of measurement (kOe) ', 'number_of_unique_magnetic_sites', 'number_of_magnetic_sites', 'Cas rn (core_1)', 'Cas rn (core_2)' ]
df = df.drop(columns=del_col)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 533 non-null    int64  
 1   crystal_structure(core/shell)      532 non-null    object 
 2   Space group(core)                  532 non-null    object 
 3   Space group(shell)                 533 non-null    object 
 4   chemical formula                   533 non-null    object 
 5   Structure                          533 non-null    object 
 6   length (D)                         521 non-null    float64
 7   width                              521 non-null    float64
 8   depth                              509 non-null    float64
 9   shape                              482 non-null    object 
 10  core_1                             533 non-null    object 
 11  core_2(shell)                      533 non-null    object 

In [121]:
#теперь можно удалить эти колонки, так как они имеют такие же значения как и crystal_structure(core/shell) и chemical formula
print(f'Unique cores: {df["core_1"].unique()}', f'Amount of unique cores composition: {len(df["core_1"].unique())}', sep='\n')
print(f'Unique shells: {df["core_2(shell)"].unique()}', f'Amount of unique shells composition: {len(df["core_2(shell)"].unique())}', sep='\n')

Unique cores: ['Zn1O1' 'Cu1O1' 'Mn0.50Zn0.50Fe2O4' 'La1Fe1O3' 'Ni1' 'Mg1Fe2O4'
 'Co1Fe2O4' 'Bi1Fe1O3' 'Co1' 'Fe1Rh1' 'Mn1O2' 'Ag1' 'Fe1' 'Fe1O1' 'Ni1O1'
 'Fe3O4' 'Bi2Fe4O9' 'Ti1O2' 'Co0.4Fe2.6O4' 'Co0.7Fe2.3O4' 'Fe0.7Co0.3'
 'Fe0.5Co0.5' 'Fe0.3Co0.7' 'La2Fe1Mn1O6' 'Fe1V1O4' 'Au1'
 'Bi0.9Gd0.1Fe0.9Ti0.1O3' 'Nd0.78Sr0.22CoO3' 'Fe2O3' 'La0.85Sr0.15FeO3'
 'Mn3O4' 'La0.67Sr0.33MnO3' 'Mn1O1' 'Co3O4' 'Ni0.99Fe0.01O1'
 'Ni0.98Fe0.02O1' 'Ni0.96Fe0.04O1' 'Ni0.97Fe0.03O1' 'Ag0.5Cr2.5O4'
 'Ag1Cr1O2' 'Co1O1' 'Mn1Fe2O4' 'Bi0.9La0.1FeO3' 'Nd0.90Ho0.10FeO3'
 'Dy1Cr1Ti1O5' 'Gd1Cr1Ti1O5' 'Ba1Fe12O19' 'Ni1Fe2O4' 'Co1Cr2O4' 'Fe1Ni1'
 'Mn1' 'Fe1F2' 'Fe5C2' 'Fe1Ni3' 'Fe2.2C1' 'La2Fe2O6' 'Co1P1']
Amount of unique cores composition: 57
Unique shells: ['Co1Fe2O4' '0' 'Ni1O1' 'Bi1Fe1O3' 'Co1O1' 'Fe3O4' 'Fe2O3' 'Bi2Fe4O9'
 'Сu1O1' 'Si1O2' 'Co3O4' 'Mn3O4' 'Ni1Fe2O4' 'Co1Mn2O4' 'Fe1' 'Mn1O1'
 'Fe3C1' 'Mg1O1' 'Mg1O1, Co1Fe2O4' 'Mo1' 'Co1']
Amount of unique shells composition: 21


In [122]:
df = df.drop(columns=['core_1', 'core_2(shell)']) #удалим их

In [123]:
#переименуем название колонк для удобства и преобразуем их в нижний регистр
df.columns = df.columns.str.lower()

new_column_names = {
    'id': 'id', 'crystal_structure(core/shell)': 'crystal_structure',
    'space group(core)': 'space_group_core', 'space group(shell)': 'space_group_shell',
    'chemical formula': 'chemical_formula', 'structure': 'structure',
    'length (d)': 'length_d_nm', 'width': 'width_nm',
    'depth': 'depth_nm', 'shape': 'shape', 'temperature (k)': 'temperature_k',
    'h-range max(koe)': 'h_range_max_koe', 'ms (emu/g)': 'sat_em_g',
    'hc (oe)': 'coer_oe', 'mrem (emu/g)': 'rem_emu_g', 'exchange bias shift hb or heb(oe)': 'exc_bias_oe',
    'vertical loop shift m vsl (emu/g)': 'ver_shift_emu_g', 'fc field(t)': 'fc_field_t'
}

# Переименование столбцов
df = df.rename(columns=new_column_names)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 533 non-null    int64  
 1   crystal_structure  532 non-null    object 
 2   space_group_core   532 non-null    object 
 3   space_group_shell  533 non-null    object 
 4   chemical_formula   533 non-null    object 
 5   structure          533 non-null    object 
 6   length_d_nm        521 non-null    float64
 7   width_nm           521 non-null    float64
 8   depth_nm           509 non-null    float64
 9   shape              482 non-null    object 
 10  temperature_k      528 non-null    float64
 11  h_range_max_koe    526 non-null    float64
 12  sat_em_g           420 non-null    object 
 13  coer_oe            510 non-null    object 
 14  mr (emu/g)         430 non-null    float64
 15  exc_bias_oe        500 non-null    float64
 16  ver_shift_emu_g    441 non

#Difference
#composition - core+shell with Ni1O1 like formula
#chemical formula - Ni50NiO50
#Structure - alhpa, gamma

In [124]:
chemical_formulas = []

for formula in df['chemical_formula']: # Итерация по каждой формуле в столбце 'chemical formula'
    if not formula.isalnum(): # Проверка, содержит ли формула только буквенно-цифровые символы
        line = re.split("-|,|/|@|–", formula) # Разделение формулы по символам '-', '@', '/', and ','
        components = [] # Создание пустого списка для хранения компонентов формулы
        for i in line:  # Итерация по каждому компоненту в разделенной формуле
            if i.strip(): # Добавление компонента в список, если он не пустой
                components.append(i.strip())
        chemical_formulas.append(components) # Добавление списка компонентов в список химических формул
    else:
        chemical_formulas.append([formula])  # Если формула состоит только из буквенно-цифровых символов, добавляем ее как есть

df['chemical_formula'] = chemical_formulas # Замена столбца новыми значениями

In [125]:
# Функция для разбивки списка в каждой ячейке на отдельные элементы и вставки их в новые столбцы
def split_elements(row):
    # Получаем список элементов из ячейки
    elements = row['chemical_formula']
    # Размещаем элементы в соответствующих столбцах
    for i, element in enumerate(elements):
        if i < 3:
            row[f'c{i+1}'] = element
    return row

# Применяем функцию к каждой строке датафрейма
df = df.apply(split_elements, axis=1)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   c1                 533 non-null    object 
 1   c2                 322 non-null    object 
 2   c3                 9 non-null      object 
 3   chemical_formula   533 non-null    object 
 4   coer_oe            510 non-null    object 
 5   crystal_structure  532 non-null    object 
 6   depth_nm           509 non-null    float64
 7   exc_bias_oe        500 non-null    float64
 8   fc_field_t         508 non-null    float64
 9   h_range_max_koe    526 non-null    float64
 10  id                 533 non-null    int64  
 11  length_d_nm        521 non-null    float64
 12  mr (emu/g)         430 non-null    float64
 13  sat_em_g           420 non-null    object 
 14  shape              482 non-null    object 
 15  space_group_core   532 non-null    object 
 16  space_group_shell  533 non

In [126]:
#проверим как разделились значения списка
print(df['c1'], df['c2'], df['c3'], end='\n')

0                    ZnO
1                    CoO
2      Mn0.50Zn0.50Fe2O4
3      Mn0.50Zn0.50Fe2O4
4      Mn0.50Zn0.50Fe2O4
             ...        
528               LaFeO3
529               LaFeO3
530               LaFeO3
531               LaFeO3
532               LaFeO3
Name: c1, Length: 533, dtype: object 0      CoFe2O4
1      CoFe2O4
2          NaN
3          NaN
4          NaN
        ...   
528        NiO
529        NiO
530        NiO
531        NiO
532        NiO
Name: c2, Length: 533, dtype: object 0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
528    NaN
529    NaN
530    NaN
531    NaN
532    NaN
Name: c3, Length: 533, dtype: object


In [127]:
print(df['c2'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 533 entries, 0 to 532
Series name: c2
Non-Null Count  Dtype 
--------------  ----- 
322 non-null    object
dtypes: object(1)
memory usage: 4.3+ KB
None


In [128]:
#Сделаем костыль для нестеохео соединений 
print(df['c1'].unique(), df['c2'].unique(), df['c3'].unique(), end='\n')
non_steh = ['Mn0.50Zn0.50Fe2O4',  'Co0.4Fe2.6O4', 'Co0.7Fe2.3O4', 'Fe0.7Co0.3', 'Fe0.5Co0.5',
 'Fe0.3Co0.7', 'Bi0.9Gd0.1Fe0.9Ti0.1O3', 'Nd0.78Sr0.22CoO3' 'Cu0.98' 'Cu0.96', 'La0.85Sr0.15FeO3', 
 'La0.67Sr0.33MnO3', 'Ni0.99Fe0.01O', 'Ni0.98Fe0.02O', 'Ni0.96Fe0.04O', 'Ni0.97Fe0.03O', 'Ag0.5Cr2.5O4',
 'Bi0.9La0.1FeO3', 'Fe2.2C', 'Fe0.02O', 'Fe0.04O']

['ZnO' 'CoO' 'Mn0.50Zn0.50Fe2O4' 'LaFeO3' 'Ni' 'MgFe2O4' 'CoFe2O4'
 'BiFeO3' 'Co' 'FeRh' 'MnO2' 'Ag' 'Fe' 'FeO' 'NiO' 'Fe3O4' 'Bi2Fe4O9'
 'TiO2' 'Co0.4Fe2.6O4' 'Co0.7Fe2.3O4' 'Fe0.7Co0.3' 'Fe0.5Co0.5'
 'Fe0.3Co0.7' 'La2FeMnO6' 'FeVO4' 'Au' 'Bi0.9Gd0.1Fe0.9Ti0.1O3'
 'Nd0.78Sr0.22CoO3' 'Cu0.98' 'Cu0.96' 'Fe2O3' 'La0.85Sr0.15FeO3' 'Mn3O4'
 'La0.67Sr0.33MnO3' 'MnO' 'Co3O4' 'Ni0.99Fe0.01O' 'Ni0.98Fe0.02O'
 'Ni0.96Fe0.04O' 'Ni0.97Fe0.03O' 'Ag0.5Cr2.5O4' 'AgCrO2' 'MnFe2O4'
 'Bi0.9La0.1FeO3' 'NdFeO3' 'DyCrTiO5' 'GdCrTiO5' 'BaFe12O19' 'NiFe2O4'
 'CoCr2O4' 'FeNi' 'Mn' 'Fe5C2' 'FeNi3' 'Fe2.2C' 'La2Fe2O6' 'FeF2' 'CoP'] ['CoFe2O4' nan 'NiO' 'BiFeO3' 'CoO' 'Fe3O4' 'Fe2O3' 'Bi2Fe4O9' 'Fe0.02O'
 'Fe0.04O' 'Co3O4' 'Mn3O4' 'NiFe2O4' 'CoMn2O4' 'MnO' 'Fe3C' 'MgO' 'Mo'
 'Co' 'Fe'] [nan 'CoFe2O4']


In [181]:
# Костыль для таких соединений
non_stoich_compounds_dict = {
    'Mn0.50Zn0.50Fe2O4': 'MnZnFe4O8', 'Co0.4Fe2.6O4': 'Co2Fe13O20',
    'Co0.7Fe2.3O4': 'Co7Fe23O40', 'Fe0.7Co0.3': 'Fe7Co3', 'Fe0.5Co0.5': 'FeCo',
    'Fe0.3Co0.7': 'Fe3Co7', 'Bi0.9Gd0.1Fe0.9Ti0.1O3': 'Bi9GdFe9TiO30',
    'Nd0.78Sr0.22CoO3': 'Nd39Sr11Co50O150', 'Cu0.98': 'Cu98', 'Cu0.96': 'Cu96',
    'La0.85Sr0.15FeO3': 'La17Sr3Fe20O60', 'La0.67Sr0.33MnO3': 'La67Sr33Mn100O300',
    'Ni0.99Fe0.01O': 'Ni99FeO100', 'Ni0.98Fe0.02O': 'Ni49FeO50', 'Ni0.96Fe0.04O': 'Ni24FeO25',
    'Ni0.97Fe0.03O': 'Ni97Fe3O100', 'Ag0.5Cr2.5O4': 'AgCr5O8', 'Bi0.9La0.1FeO3': 'Bi9LaFe10O30',
    'Fe2.2C': 'Fe11C5', 'Fe0.02O': 'FeO50', 'Fe0.04O': 'FeO25'
}

In [ ]:
def replace_non_stoichiometric(column, non_stoich_compounds_dict):
    '''Функция заменяет значения ключей на значения для этих ключей'''
    for i, value in enumerate(column):
        if value != 0 and value in non_stoich_compounds_dict:
            column.at[i] = non_stoich_compounds_dict[value]
    return column

In [184]:
# Применение функции к каждому столбцу
new_df_c1 = replace_non_stoichiometric(df['c1'], non_stoich_compounds_dict)
new_df_c2 = replace_non_stoichiometric(df['c2'], non_stoich_compounds_dict)
new_df_c3 = replace_non_stoichiometric(df['c3'], non_stoich_compounds_dict)

# Вывод уникальных значений после применения функции
unique_values_c1 = new_df_c1.unique()
unique_values_c2 = new_df_c2.unique()
unique_values_c3 = new_df_c3.unique()

print("Уникальные значения в столбце c1 после применения функции:", unique_values_c1)
print("Уникальные значения в столбце c2 после применения функции:", unique_values_c2)
print("Уникальные значения в столбце c3 после применения функции:", unique_values_c3)

Уникальные значения в столбце c1 после применения функции: ['ZnO' 'CoO' 'MnZnFe4O8' 'LaFeO3' 'Ni' 'MgFe2O4' 'CoFe2O4' 'BiFeO3' 'Co'
 'FeRh' 'MnO2' 'Ag' 'Fe' 'FeO' 'NiO' 'Fe3O4' 'Bi2Fe4O9' 'TiO2'
 'Co2Fe13O20' 'Co7Fe23O40' 'Fe7Co3' 'FeCo' 'Fe3Co7' 'La2FeMnO6' 'FeVO4'
 'Au' 'Bi9GdFe9TiO30' 'Nd39Sr11Co50O150' 'Cu98' 'Cu96' 'Fe2O3'
 'La17Sr3Fe20O60' 'Mn3O4' 'La67Sr33Mn100O300' 'MnO' 'Co3O4' 'Ni99FeO100'
 'Ni49FeO50' 'Ni24FeO25' 'Ni97Fe3O100' 'AgCr5O8' 'AgCrO2' 'MnFe2O4'
 'Bi9LaFe10O30' 'NdFeO3' 'DyCrTiO5' 'GdCrTiO5' 'BaFe12O19' 'NiFe2O4'
 'CoCr2O4' 'FeNi' 'Mn' 'Fe5C2' 'FeNi3' 'Fe11C5' 'La2Fe2O6' 'FeF2' 'CoP']
Уникальные значения в столбце c2 после применения функции: ['CoFe2O4' nan 'NiO' 'BiFeO3' 'CoO' 'Fe3O4' 'Fe2O3' 'Bi2Fe4O9' 'FeO50'
 'FeO25' 'Co3O4' 'Mn3O4' 'NiFe2O4' 'CoMn2O4' 'MnO' 'Fe3C' 'MgO' 'Mo' 'Co'
 'Fe']
Уникальные значения в столбце c3 после применения функции: [nan 'CoFe2O4']


## Теперь поработаем с space_group_core и space_group_shell

In [129]:
#посмотрим какие структуры у нас есть
print(f"Core unique values: {df['space_group_core'].unique()}")
print(f"Core unique values: {df['space_group_shell'].unique()}")

core unique values: ['p63mc' 'fm3m' 'fd3m' 'pbnm' 'r3c' 'p63/mmc' 'pm3m' 'i4/m' 'im3m' 'pbam'
 'p42/mnm' 'pnma' 'p1' nan 'r3с' 'i41/amd' 'P63/mmc' 'p3m1' 'r3m' 'p4332'
 'p63mmc' 'p4/mmm' 'i43m' 'cmcm' 'p4132' 'c2/c' 'p6322']
core unique values: ['fd3m' '0' 'fm3m' 'r3c' 'p4332' 'cc' 'pbam' 'p4132' 'pnma' 'i41/amd'
 'p41212' 'im3m' 'p63/mmc' 'p42/mnm']


In [187]:
#создадим словарь, чтобы перевести простаранственные группы в числовые обозначения пространнственных групп
dict_space_group = {
    1: "P1", 2: "P1", 3: "P121", 4: "P1211", 5: "C121", 6: "P1m1", 7: "P1c1", 8: "C1m1",
    9: "C1c1", 10: "P12/m1", 11: "P121/m1", 12: "C12/m1", 13: "P12/c1", 14: "P121/c1",
    15: "C12/c1", 16: "P222", 17: "P2221", 18: "P21212", 19: "P212121", 20: "C2221",
    21: "C222", 22: "F222", 23: "I222", 24: "I212121", 25: "Pmm2", 26: "Pmc21", 27: "Pcc2",
    28: "Pma2", 29: "Pca21", 30: "Pnc2", 31: "Pmn21", 32: "Pba2", 33: "Pna21", 34: "Pnn2",
    35: "Cmm2", 36: "Cmc21", 37: "Ccc2", 38: "Amm2", 39: "Abm2", 40: "Ama2", 41: "Aba2",
    42: "Fmm2", 43: "Fdd2", 44: "Imm2", 45: "Iba2", 46: "Ima2", 47: "Pmmm", 48: "Pnnn",
    49: "Pccm", 50: "Pban", 51: "Pmma", 52: "Pnna", 53: "Pmna", 54: "Pcca", 55: "Pbam",
    56: "Pccn", 57: "Pbcm", 58: "Pnnm", 59: "Pmmn", 60: "Pbcn", 61: "Pbca", 62: "Pnma",
    63: "Cmcm", 64: "Cmca", 65: "Cmmm", 66: "Cccm", 67: "Cmma", 68: "Ccca", 69: "Fmmm",
    70: "Fddd", 71: "Immm", 72: "Ibam", 73: "Ibca", 74: "Imma", 75: "P4", 76: "P41",
    77: "P42", 78: "P43", 79: "I4", 80: "I41", 81: "P4", 82: "I4", 83: "P4/m", 84: "P42/m",
    85: "P4/n", 86: "P42/n", 87: "I4/m", 88: "I41/a", 89: "P422", 90: "P4212", 91: "P4122",
    92: "P41212", 93: "P4222", 94: "P42212", 95: "P4322", 96: "P43212", 97: "I422",
    98: "I4122", 99: "P4mm", 100: "P4bm", 101: "P42cm", 102: "P42nm", 103: "P4cc",
    104: "P4nc", 105: "P42mc", 106: "P42bc", 107: "I4mm", 108: "I4cm", 109: "I41md",
    110: "I41cd", 111: "P42m", 112: "P42c", 113: "P421m", 114: "P421c", 115: "P4m2",
    116: "P4c2", 117: "P4b2", 118: "P4n2", 119: "I4m2", 120: "I4c2", 121: "I42m",
    122: "I42d", 123: "P4/mmm", 124: "P4/mcc", 125: "P4/nbm", 126: "P4/nnc", 127: "P4/mbm",
    128: "P4/mnc", 129: "P4/nmm", 130: "P4/ncc", 131: "P42/mmc", 132: "P42/mcm",
    133: "P42/nbc", 134: "P42/nnm", 135: "P42/mbc", 136: "P42/mnm", 137: "P42/nmc",
    138: "P42/ncm", 139: "I4/mmm", 140: "I4/mcm", 141: "I41/amd", 142: "I41/acd",
    143: "P3", 144: "P31", 145: "P32", 146: "R3", 147: "P3", 148: "R3", 149: "P312",
    150: "P321", 151: "P3112", 152: "P3121", 153: "P3212", 154: "P3221", 155: "R32",
    156: "P3m1", 157: "P31m", 158: "P3c1", 159: "P31c", 160: "R3m", 161: "R3c", 162: "P31m",
    163: "P31c", 164: "P3m1", 165: "P3c1", 166: "R3m", 167: "R3c", 168: "P6", 169: "P61",
    170: "P65", 171: "P62", 172: "P64", 173: "P63", 174: "P6", 175: "P6/m", 176: "P63/m",
    177: "P622", 178: "P6122", 179: "P6522", 180: "P6222", 181: "P6422", 182: "P6322",
    183: "P6mm", 184: "P6cc", 185: "P63cm", 186: "P63mc", 187: "P6m2", 188: "P6c2",
    189: "P62m", 190: "P62c", 191: "P6/mmm", 192: "P6/mcc", 193: "P63/mcm", 194: "P63/mmc",
    195: "P23", 196: "F23", 197: "I23", 198: "P213", 199: "I213", 200: "Pm3", 201: "Pn3",
    202: "Fm3", 203: "Fd3", 204: "Im3", 205: "Pa3", 206: "Ia3", 207: "P432", 208: "P4232",
    209: "F432", 210: "F4132", 211: "I432", 212: "P4332", 213: "P4132", 214: "I4132",
    215: "P43m", 216: "F43m", 217: "I43m", 218: "P43n", 219: "F43c", 220: "I43d",
    221: "Pm3m", 222: "Pn3n", 223: "Pm3n", 224: "Pn3m", 225: "Fm3m", 226: "Fm3c",
    227: "Fd3m", 228: "Fd3c", 229: "Im3m", 230: "Ia3d"
}


In [ ]:
# Создание словаря со значениями в нижнем регистре для сравнени по нему
dict_space_group_lower = {key: value.lower() for key, value in dict_space_group.items()}

In [140]:
# Проход по каждой ячейке в колонке 'Space group(core)' и применение процедуры
for index, row in df.iterrows():
    space_group_core = row['space_group_core']
    if space_group_core.lower() in dict_space_group:
        df.at[index, 'space_group_core'] = dict_space_group[space_group_core.lower()] + ": \"" + dict_space_group[space_group_core.lower()] + "\""
    else:
        df.at[index, 'space_group_core'] = "Unknown"

print(df)

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
dict_crystal_structure = {
    "triclinic":[1, 2], 
    "monoclinic":[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    "orthorhombic":[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74], 
    "tetragonal":[75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142], 
    "trigonal":[143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167], 
    "hexagonal":[168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194],
    "cubic":[195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230]
}               